In [1]:
import uproot as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
run="run1"
dm_type = 'fermion'
target_pot=0
signal_pot={}

signal_correction = { "0.01":0.94903385, "0.02":1.03561568, "0.03":1.02326517, "0.04":1.00641579, 
                     "0.05":1.00650082, "0.06":1.01293082, "0.07":1.07433777, 
                     "0.08":1.10175345, "0.09":1.0415591 , "0.10":1.1014797}
    

    
signal_pot = { "0.01":3.9e19, "0.02":3.4e+20, "0.03":1.49e+21, 
                  "0.04":5.59e+21, "0.05":1.89e+22, "0.06":6.02e+22,
                  "0.07":1.8e+23, "0.08":5.6e+23, 
                  "0.09":1.2e+24, "0.10":2.9e+24}

''' 
signal_pot = { "0.01":1.06e20, "0.02":9.44e+20, "0.03":4.02e+21, 
                  "0.04":1.5e+22, "0.05":5.03e+22, "0.06":1.57e+23,
                  "0.07":4.6e+23, "0.08":1.37e+24, 
                  "0.09":2.77e+24, "0.10":6.62e+24}
                  
'''


scalar_factors = np.array([0.04444236, 0.03876295, 0.03149615, 0.03032621, 0.02560909, 0.02296708,
 0.0244469, 0.02523209, 0.03118899, 0.03338144, 0.02626005, 0.01194017,
 0.0040589 ])

    
    
if(run=="run1"):
    target_pot = 2.38e20
    
else:
    target_pot = 5.18e20 

    
detvar_uncert = 0.25 
xsec_uncert = 0.2 

In [3]:
print("Selected pot: ", target_pot)

Selected pot:  2.38e+20


In [4]:
def logit_transform(score):
    return np.log(score/(1-score))

def filter_df(df):
    df = df[df['signal_score'] >= 0.5]
    return df

def GetStatsUncert(df_a, weight_array,my_custom_bins,plot_var):
    (counts_df, bins_df) = np.histogram(df_a[plot_var], bins = my_custom_bins, weights=np.square(weight_array))
    return counts_df

In [5]:
def MakeSignalHist():
    base_dir = "/home/lmlepin/Desktop/dm_sets/dark_tridents_analysis/run1_samples/"
    signal_file = u.recreate(base_dir + dm_type + "_signal_hist_" + run + "_CNN_old.root")
    masses = ["0.01", "0.02", "0.03", "0.04", "0.05", "0.06", "0.07", "0.08", "0.09", "0.10","0.2","0.3","0.4"]
    counter = 0 
    for mass in masses:
        if(float(mass) <= 0.1):          
            df_signal = pd.read_csv(base_dir + "dt_overlay_"+ mass + "_CNN_scores_8441_steps.csv")
            df_signal = df_signal.drop_duplicates()
            df_signal = df_signal[df_signal['n_tracks'] == 0]
            signal_scaling = (signal_correction[mass]*target_pot/signal_pot[mass])
            if(dm_type == "scalar"):
                signal_scaling*=scalar_factors[counter]
        else:
            df_signal = pd.read_csv(base_dir + "run1_dt_{}_eta_new_CNN_scores_8441_steps.csv".format(mass))
            df_signal = df_signal.drop_duplicates()
            signal_fraction = len(df_signal)/len(df_signal[df_signal['signal_score']>= 0.])
            signal_scaling = (7.6*signal_fraction*target_pot)/df_signal['total_pot'][0]
            if(dm_type == "scalar"):
                signal_scaling*=scalar_factors[counter]
            
        df_signal = filter_df(df_signal)
        df_signal['signal_score'] = logit_transform(df_signal['signal_score'])
        my_custom_bins = [ 0., 1.375, 2.75, 4.125, 5.5, 11.] # Last four bins merged bc of low stats
        (counts_signal, bins_signal) = np.histogram(df_signal['signal_score'], bins = my_custom_bins )
        stats_uncert = np.sqrt(counts_signal)*signal_scaling
        print("Mass: " + mass)
        print("Total scaled events: ", np.sum(counts_signal*signal_scaling))
        print("MC uncertainty: ",stats_uncert)
        print("CV bins: ", counts_signal*signal_scaling)
        print("MC fraction error: ", (stats_uncert/(counts_signal*signal_scaling))*100)
        print("\n")
        signal_file["signal_"+mass] = (counts_signal*signal_scaling, bins_signal)
        signal_file["signal_mc_error_"+mass] = (stats_uncert, bins_signal)
        counter+=1 
        
        
        
        '''
        if(mass == "0.05"):
            collie_file = u.update(base_dir + "dark_trident_hist_for_collie_" + run + "_CNN.root")
            collie_file['theta'] = ((1e-3)*counts_signal/counts_signal,bins_signal)
            collie_file['signal'] = (counts_signal*signal_scaling, bins_signal)
            collie_file['signal_stat_uncertainty'] = (stats_uncert/(counts_signal*signal_scaling), bins_signal)
            collie_file['signal_DetVar_uncertainty'] = ((counts_signal*signal_scaling*detvar_uncert),bins_signal)
            collie_file['signal_DetVar_uncertainty_frac'] = ((counts_signal*signal_scaling*detvar_uncert)/(counts_signal*signal_scaling),bins_signal)
            collie_file['signal_xsec_uncertainty'] = ((counts_signal*signal_scaling*xsec_uncert),bins_signal)
            collie_file['signal_xsec_uncertainty_frac'] = ((counts_signal*signal_scaling*xsec_uncert)/(counts_signal*signal_scaling),bins_signal)
        '''

In [6]:
MakeSignalHist()

Mass: 0.01
Total scaled events:  33312.93753429743
MC uncertainty:  [167.45464227 203.11715684 222.27746057 208.57585913 175.28380409]
CV bins:  [4841.72736069 7123.59408331 8530.93828025 7511.62725695 5305.0505531 ]
MC fraction error:  [3.45857232 2.85132974 2.60554529 2.77670672 3.304093  ]


Mass: 0.02
Total scaled events:  4164.72845712
MC uncertainty:  [20.97294348 24.8493819  27.25007728 26.07737039 23.22043427]
CV bins:  [ 606.76722691  851.7938968  1024.32746909  938.06068294  743.77918138]
MC fraction error:  [3.45650565 2.91729983 2.66028961 2.77992361 3.12195271]


Mass: 0.03
Total scaled events:  906.1541881813691
MC uncertainty:  [4.56191941 5.47000712 5.97423235 5.84539889 5.24562805]
CV bins:  [127.32577788 183.06145216 218.36616079 209.04964046 168.3511569 ]
MC fraction error:  [3.58287182 2.98807152 2.73587828 2.79617744 3.11588476]


Mass: 0.04
Total scaled events:  245.3115983299642
MC uncertainty:  [1.18282386 1.43912044 1.62035663 1.5443551  1.42501736]
CV bins:  [